In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")

In [3]:
df.head()

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.121,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3769828.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.118,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3787706.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.116,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3806634.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.115,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3825655.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.114,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

## Step 1. Minor data processing

In [5]:
# First, we fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!= 0, df['gdp']/df['population'],0)

In [6]:
# make DataFrame pipeline interactive
idf = df.interactive()

## Step 2. Create filters for time and continents

In [17]:
# define panel widget
# step defines the number of year shifted for every slide of the slider
# value defines the default value when you run the code
year_slider = pn.widgets.IntSlider(name = 'Year slider', start = 1750, end = 2020, step = 1, value = 1850)
year_slider

IntSlider(end=2020, name='Year slider', start=1750, value=1850)

In [18]:
# this list is based of our data
continents = ['Asia','Oceania','Europe','Africa','North America','South America', 'Antarctica']

checkbox_group = pn.widgets.CheckBoxGroup(
    name='Select continents', value=continents, options=continents,
    inline=True)

checkbox_group

CheckBoxGroup(inline=True, name='Select continents', options=['World', 'Asia', ...], value=['World', 'Asia', ...])

## Step 3. Create a plot for CO2 and CO2 per capita emission by continent 

In [20]:
# radio buttons for CO2 measures
# this line of code sets the Y axis for our chart
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['co2','co2_per_capita',],
    button_types = 'success'
)

yaxis_co2

RadioButtonGroup(name='Y axis', options=['co2', 'co2_per_capita'], value='co2')

In [21]:
# here we are creating a data pipeline and connect said pipeline to our widgets
co2_pipeline = (
    # idf is our interactive data pipeline
    # idf.year <= year_slide is to select all year data less and equal than the year in the slider
    # idf.country.isin(checkbox_group) is connect our continents filter with the data shown in the plot
    idf[(idf.year <= year_slider) & (idf.country.isin(checkbox_group))]
    .groupby(['country','year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by=['country','year'])
    .reset_index(drop=True)
)

In [22]:
co2_plot = co2_pipeline.hvplot(x='year', by='country', y = yaxis_co2, line_widthe = 2, title = 'CO2 emission by continent')
co2_plot

## Step 3. Prepare table for CO2 emission over time by continent

In [23]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10,sizing_mode='stretch_width')
co2_table

## Step 4. Create scatterplot for CO2 vs GDP

In [24]:
# to solve the previous problem about showing aggregate, we can groupby our data and find the mean
co2_vs_gdp_scatterplot_pipeline = (
    idf[(idf.year == year_slider) & (~ (idf.country.isin(checkbox_group)))]
    .groupby(['country','year','gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by=['country','year'])
    .reset_index(drop=True)
)

In [25]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', 
                                                                y='co2', 
                                                                by='country', 
                                                                size=80, 
                                                                kind='scatter',
                                                                alpha=0.7,
                                                                legend=False,
                                                                height=500,
                                                                width=500)
co2_vs_gdp_scatterplot

## Step 5. Create bar chart for CO2 sources by continent

In [14]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['coal_co2','oil_co2','gas_co2'],
    button_type='success')

# continents_excl_world = continents.copy()
# continents_excl_world.remove("World")

co2_source_bar_pipeline = (
    idf[(idf.year == year_slider) & (idf.country.isin(checkbox_group))]
    .groupby(['year','country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [15]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar',
                                                     x='country',
                                                     y=yaxis_co2_source,
                                                     title='CO2 source by continent')
co2_source_bar_plot                                        

## Last step. Combine everything into a dashboard

In [ ]:
# panel has a lot of pre designed template that we can use right away
template = pn.template.FastListTemplate(
    title='World CO2 emission dashboard',
    sidebar=[pn.pane.Markdown("# CO2 Emission and Climate Change"),
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
             pn.pane.PNG('planet_globe.png', sizing_mode='scale_both')
            ],
    # add all the widgets to specific rows
    # pn.Column define the number of columns and 
    # set plot size to fit the template
    # sizing_mode='stretch_both' will make the plot fit the size of each column
    # width_policy='max' will make each column to fit the size of screen
    main=[pn.Row(pn.Column(pn.pane.Markdown("## Select year (up to)"),year_slider,width_policy='max'),pn.Column(pn.pane.Markdown("## Select continents"),checkbox_group,width_policy='max')),
          pn.Row(pn.Column(yaxis_co2, co2_plot.panel(sizing_mode='stretch_both'),width_policy='max'), #left hand side column
                 co2_table.panel(sizing_mode='stretch_both'),width_policy='max'), # right hand side column
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(sizing_mode='stretch_both'),width_policy='max'), #left hand side column
                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(sizing_mode='stretch_both'),width_policy='max'))], #right hand side column
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

# template.show()
template.servable();